In [1]:
from omegaconf import OmegaConf
import torch
from utils.utils import instantiate_from_config
from model.cldm.model import load_state_dict
import os
import random

config_path = "configs/v1.yaml"

config = OmegaConf.load(config_path)

model = instantiate_from_config(config.model)
model.load_state_dict(load_state_dict("checkpoints/epoch=32-step=124938-v1.ckpt", "cpu"), strict=False)
model = model.cuda()

c:\Users\bscharnagl\anaconda3\envs\fittingathome\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No module 'xformers'. Proceeding without it.
ControlLDM: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


d:\Repos\size_aware_tryon\model\ldm\models\diffusion\ddpm.py:624: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt, map_location="cpu")["s

First Stage restored from checkpoints/vae-ft-mse-840000-ema-pruned.ckpt


c:\Users\bscharnagl\anaconda3\envs\fittingathome\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
d:\Repos\size_aware_tryon\model\cldm\model.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer b

Loaded state_dict from [checkpoints/epoch=32-step=124938-v1.ckpt]


In [ ]:
from utils.utils import read_image, read_keypoints, draw_pose, draw_landmarks
import numpy as np
import cv2
from torchvision.utils import save_image

prompt = "model posing"

root = "sample_images"

fk_jsons = os.listdir(os.path.join(root, "fashion_keypoints_posed"))
hk_jsons = os.listdir(os.path.join(root, "human_keypoints_posed"))

offsets = [0.0,30,-30]

for i in fk_jsons:
    id = i[:-7]
    print(id)
    for offset in offsets:
        fk_tensor = read_keypoints(root, "fashion_keypoints_posed", id)
        hk_tensor = read_keypoints(root, "human_keypoints_posed", id)
    
        blank_image = np.zeros((1024,768,3), np.uint8)
        img,m_r,m_l = draw_pose(blank_image, hk_tensor)
        img = draw_landmarks(img, fk_tensor, offset, m_r, m_l)
        cv2.imwrite(root+"/output/"+str(id)+"_keypoints_"+str(offset)+".png", img)

        batch = {
            "txt": prompt,
            "human_image": read_image(root+"/human_image/"+id+"_0.jpg", (512,512)),
            "keypoints_vis": read_image(root+"/output/"+str(id)+"_keypoints_"+str(offset)+".png", (512,512)),
            "mask": read_image(root+"/cloth_agnostic_mask/"+id+"_0.png", (64,64)),
        }

        from torch.utils.data import default_collate
        for k in batch.keys():
            batch[k] = default_collate(batch[k])
            if isinstance(batch[k], torch.Tensor):
                batch[k] = batch[k].unsqueeze(0).cuda()
            else:
                batch[k] = [batch[k]]
        
        images = model.log_images(batch)

        for k in images:
            if k == "reconstruction" or k == "control":
                continue
            images[k] = (images[k] + 1.0) / 2.0
            batch["keypoints_vis"][0] = (batch["keypoints_vis"][0] + 1.0) / 2.0
            
            save_image(images[k], root+"/output/"+str(id)+"_"+k+"_"+str(offset)+".jpg")

013582
torch.Size([1, 1, 64, 64]) tensor(-1., device='cuda:0') tensor(1., device='cuda:0')
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [02:29<00:00,  2.99s/it]


Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [05:02<00:00,  6.05s/it]


torch.Size([1, 1, 64, 64]) tensor(-1., device='cuda:0') tensor(1., device='cuda:0')
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


...

DDIM Sampler: 100%|██████████| 50/50 [05:27<00:00,  6.55s/it]


torch.Size([1, 1, 64, 64]) tensor(-1., device='cuda:0') tensor(1., device='cuda:0')
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [02:30<00:00,  3.01s/it]


Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler:   4%|▍         | 2/50 [00:11<04:47,  5.99s/it]